In [ ]:
import panel as pn
import sys
from wbib import wbib
import requests
from pybtex.database.input import bibtex
from tqdm import tqdm
import time

pn.extension()



In [ ]:
file_input = pn.widgets.FileInput(filename="bibtex.bib")
if file_input.value is not None:
    file_input.save('bibtex.bib')

pn.Column(
    pn.widgets.FileDownload(file='test.bib', embed=True),
    pn.Row(file_input),
    pn.widgets.FileDownload(file='error_log.txt', embed=True)
).servable()